In [4]:
import numpy as np
import scipy as sp
from mgwr.sel_bw import Sel_BW
from mgwr.gwr import GWR, MGWR
import pandas as pd
import libpysal as ps
import pickle as pk
import os

In [5]:
data = ps.io.open(ps.examples.get_path('GData_utm.csv'))
coords = list(zip(data.by_col('X'), data.by_col('Y')))
y = np.array(data.by_col('PctBach')).reshape((-1,1))
rural  = np.array(data.by_col('PctRural')).reshape((-1,1))
pov = np.array(data.by_col('PctPov')).reshape((-1,1)) 
black = np.array(data.by_col('PctBlack')).reshape((-1,1))
fb = np.array(data.by_col('PctFB')).reshape((-1,1))
pop = np.array(data.by_col('TotPop90')).reshape((-1,1))


X = np.hstack([fb, black, rural])


In [6]:
X = (X - X.mean(axis=0)) / X.std(axis=0)

y = y.reshape((-1,1))

y = (y - y.mean(axis=0)) / y.std(axis=0)

sel = Sel_BW(coords, y, X)

bw = sel.search()
print('bw:', bw)
gwr = GWR(coords, y, X, bw)
gwr_results = gwr.fit()
print('aicc:', gwr_results.aicc)
print('ENP:', gwr_results.ENP)
print('sigma2:', gwr_results.sigma2)

bw: 117.0
aicc: 299.0508086830288
ENP: 11.804769716730096
sigma2: 0.34774354749782793


In [7]:
X = (X - X.mean(axis=0)) / X.std(axis=0)

y = y.reshape((-1,1))

y = (y - y.mean(axis=0)) / y.std(axis=0)

selector = Sel_BW(coords, y, X, multi=True, constant=True)
bw = selector.search(multi_bw_min=[2], multi_bw_max=[159])
print('bw(intercept):', bw[0])
print('bw(foreign):', bw[1])
print('bw(african_amer):', bw[2])
print('bw(rural):', bw[3])
mgwr = MGWR(coords, y, X, selector, constant=True)
mgwr_results = mgwr.fit()
print('aicc:', mgwr_results.aicc)
print('sigma2:', mgwr_results.sigma2)
print('ENP(model):', mgwr_results.ENP)
print('adj_alpha(model):', mgwr_results.adj_alpha[1])
print('critical_t(model):', mgwr_results.critical_tval(alpha=mgwr_results.adj_alpha[1]))
alphas = mgwr_results.adj_alpha_j[:,1]
critical_ts = mgwr_results.critical_tval()
print('ENP(intercept):', mgwr_results.ENP_j[0])
print('adj_alpha(intercept):', alphas[0])
print('critical_t(intercept):', critical_ts[0])
print('ENP(foreign):', mgwr_results.ENP_j[1])
print('adj_alpha(foreign):', alphas[1])
print('critical_t(foreign):', critical_ts[1])
print('ENP(african_amer):', mgwr_results.ENP_j[2])
print('adj_alpha(african_amer):', alphas[2])
print('critical_t(african_amer):', critical_ts[2])
print('ENP(rural):', mgwr_results.ENP_j[3])
print('adj_alpha(rural):', alphas[3])
print('critical_t(rural):', critical_ts[3])

  4%|▍         | 9/200 [00:01<00:37,  5.10it/s]


bw(intercept): 92.0
bw(foreign): 101.0
bw(african_amer): 136.0
bw(rural): 158.0
aicc: 297.12013812258783
sigma2: 0.34477258292171475
ENP(model): 11.36825087269831
adj_alpha(model): 0.017592855949398047
critical_t(model): 2.399257840857394
ENP(intercept): 3.844671080264143
adj_alpha(intercept): 0.013005013681577365
critical_t(intercept): 2.512107491068591
ENP(foreign): 3.513770805151652
adj_alpha(foreign): 0.014229727199820033
critical_t(foreign): 2.4788879239423856
ENP(african_amer): 2.2580525278898254
adj_alpha(african_amer): 0.022142974701622884
critical_t(african_amer): 2.3106911297007184
ENP(rural): 1.7517564593926893
adj_alpha(rural): 0.028542780437261432
critical_t(rural): 2.210001836555586
